In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("/home/buhturi/Desktop/datasets/Offensive_dataset.csv")
data.head(10)

,tweet,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",1
6,"!!!!!!""@__BrighterDays: I can not just sit up ...",1
7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,1
8,""" &amp; you might not get ya bitch back &amp; ...",1
9,""" @rhythmixx_ :hobbies include: fighting Maria...",1


In [3]:
tweets = list(data['tweet'])
labels = list(data['label'])

In [4]:
test_n = int(len(tweets) * 0.20)
train_n = len(tweets) - test_n
train_n


19827

In [5]:
train_x = tweets[:train_n]
test_x = tweets[train_n:]
train_y = np.array([labels[:train_n]]).T
test_y = np.array([labels[train_n:]]).T

In [6]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer 

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/buhturi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
def process_tweet(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    stopwords_english = stopwords.words('english')
    tweet_clean = []
    for word in tweet_tokens:
        if(word not in stopwords_english and  word not in string.punctuation):
            tweet_clean.append(word)

    stemmer = PorterStemmer() 
    tweet_stem = []
    for word in tweet_clean:
        word_stem = stemmer.stem(word)
        tweet_stem.append(word_stem) 

    return tweet_stem

In [9]:
def build_freqs(tweets, ys):
    
    freqs = {}
    yslist = np.squeeze(ys).tolist();
    for y, tweet in zip(yslist, tweets):
        
        for word in process_tweet(tweet):
            
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [10]:
freqs = build_freqs(train_x, train_y)

In [11]:
def sigmoid(z): return 1/(1+np.exp(-z)) 

In [12]:
def gradiantDescent(x, y, theta, alpha, num_iters):
    m = x.shape[0]
    for i in range(0, num_iters):
        z = np.dot(x, theta)
        h = sigmoid(z)
        j = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h))) 
        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))
        j = float(j)
    return j, theta

In [13]:
def extract_features(tweet, freqs):
    word_l = process_tweet(tweet)
    x = np.zeros((1, 3))
    x[0,0] = 1
    for word in word_l:
        x[0,1] += freqs.get((word,1), 0)
        x[0,2] += freqs.get((word,0), 0)
    assert(x.shape == (1,3))
    return x

In [14]:
#train
X = np.zeros((len(train_x),3))
for i in range(len(train_x)):  
    X[i, :] = extract_features(train_x[i], freqs)
Y = train_y
j, theta = gradiantDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)

In [15]:
Y.shape

(19827, 1)

In [16]:
print(theta)

[[ 3.09936080e-08]
 [ 4.53749844e-04]
 [-2.66270056e-05]]


In [17]:
#part of test
def predict_tweet(tweet, freqs, theta):
    x = extract_features(tweet, freqs)
    y_pred = sigmoid(np.dot(x, theta))
    return y_pred

In [18]:

y_pred = predict_tweet(train_x[0], freqs, theta)
y_pred

array([[0.82710118]])

In [19]:
def test_lr(test_x, test_y, freqs, theta):
    y_hat = []
    for tweet in test_x:
        y_pred = predict_tweet(tweet, freqs, theta)
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            y_hat.append(0)
    s = 0
    for i in range(len(test_y)):
        if y_hat[i] == test_y[i]: s +=1
    accuracy = s / len(test_y) 
    #print(accuracy)
    return accuracy

In [20]:
acc = test_lr(test_x, test_y, freqs, theta)
acc

0.8399919289749799

In [26]:
my_tweet = "hey bitch!"
p = predict_tweet(my_tweet,freqs,theta)
print(p)
if(p > 0.5): print("offensive language")
else : print("not offinsive language")

[[0.98377403]]
offensive language
